In [ ]:
import time
import numpy as np
import pandas as pd

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from scipy.stats import spearmanr

d:\College\SEM-6\UCS654\Lecture\Assignment-5-Topsis-for-Pretrained-Models\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = load_dataset("sentence-transformers/stsb", split="test")

sentences1 = dataset["sentence1"]
sentences2 = dataset["sentence2"]
labels = dataset["score"]

models = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/paraphrase-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-mpnet-base-v2",
    "sentence-transformers/distilbert-base-nli-stsb-mean-tokens",
    "sentence-transformers/bert-base-nli-mean-tokens",
]

In [10]:
def evaluate_model(model_name, s1, s2, labels, sample_size=500):
    model = SentenceTransformer(model_name, device="cpu")

    s1 = s1[:sample_size]
    s2 = s2[:sample_size]
    labels = labels[:sample_size]

    start = time.time()
    emb1 = model.encode(s1, convert_to_numpy=True, show_progress_bar=False)
    emb2 = model.encode(s2, convert_to_numpy=True, show_progress_bar=False)
    end = time.time()

    cosine_sim = np.sum(emb1 * emb2, axis=1) / (
        np.linalg.norm(emb1, axis=1) * np.linalg.norm(emb2, axis=1)
    )

    spearman, _ = spearmanr(cosine_sim, labels)
    avg_time_ms = ((end - start) / sample_size) * 1000

    return spearman, avg_time_ms, model

In [11]:
from huggingface_hub import scan_cache_dir

def get_model_size_mb(repo_id):
    cache_info = scan_cache_dir()
    for repo in cache_info.repos:
        if repo.repo_id == repo_id:
            return repo.size_on_disk / (1024 * 1024)
    return None

In [12]:
results = []

for model_name in models:
    print(f"Evaluating: {model_name}")

    accuracy, time_ms, _ = evaluate_model(
        model_name,
        sentences1,
        sentences2,
        labels
    )

    size_mb = get_model_size_mb(model_name)

    results.append([
        model_name.split("/")[-1],
        accuracy,
        time_ms,
        size_mb
    ])

df = pd.DataFrame(
    results,
    columns=["Model", "Accuracy", "Time_ms", "Size_MB"]
)

print(df)


Evaluating: sentence-transformers/all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 313.84it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Evaluating: sentence-transformers/all-mpnet-base-v2


d:\College\SEM-6\UCS654\Lecture\Assignment-5-Topsis-for-Pretrained-Models\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kshitiz\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 199/199 [00:0

Evaluating: sentence-transformers/paraphrase-MiniLM-L12-v2


...

Evaluating: sentence-transformers/distilbert-base-nli-stsb-mean-tokens


d:\College\SEM-6\UCS654\Lecture\Assignment-5-Topsis-for-Pretrained-Models\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kshitiz\.cache\huggingface\hub\models--sentence-transformers--distilbert-base-nli-stsb-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████

Evaluating: sentence-transformers/bert-base-nli-mean-tokens


d:\College\SEM-6\UCS654\Lecture\Assignment-5-Topsis-for-Pretrained-Models\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kshitiz\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 199/1

                                  Model  Accuracy    Time_ms     Size_MB
0                      all-MiniLM-L6-v2  0.927632   6.343399   87.335937
1                     all-mpnet-base-v2  0.930934  49.850491  418.360636
2              paraphrase-MiniLM-L12-v2  0.917021  11.850999  127.953589
3              paraphrase-mpnet-base-v2  0.935278  40.254031  418.354918
4  distilbert-base-nli-stsb-mean-tokens  0.915163  22.171097  253.835472
5             bert-base-nli-mean-tokens  0.879270  42.581038  418.337442


In [13]:
# Decision Matrix
decision_matrix = df[["Accuracy", "Time_ms", "Size_MB"]].values

# Weights (sum = 1)
weights = np.array([0.50, 0.30, 0.20])

# Benefit (+1) / Cost (-1)
criteria_type = np.array([1, -1, -1])

# Normalize
norm_matrix = decision_matrix / np.sqrt((decision_matrix**2).sum(axis=0))

# Weighted normalization
weighted_matrix = norm_matrix * weights

# Ideal best and worst
ideal_best = np.where(
    criteria_type == 1, weighted_matrix.max(axis=0), weighted_matrix.min(axis=0)
)

ideal_worst = np.where(
    criteria_type == 1, weighted_matrix.min(axis=0), weighted_matrix.max(axis=0)
)

# Distances
dist_best = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
dist_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))

# TOPSIS score
df["TOPSIS_Score"] = dist_worst / (dist_best + dist_worst)

# Rank
df["Rank"] = df["TOPSIS_Score"].rank(ascending=False)
df = df.sort_values("Rank")

print("\nTOPSIS Ranking:\n")
print(df)


TOPSIS Ranking:

                                  Model  Accuracy    Time_ms     Size_MB  \
0                      all-MiniLM-L6-v2  0.927632   6.343399   87.335937   
2              paraphrase-MiniLM-L12-v2  0.917021  11.850999  127.953589   
4  distilbert-base-nli-stsb-mean-tokens  0.915163  22.171097  253.835472   
3              paraphrase-mpnet-base-v2  0.935278  40.254031  418.354918   
5             bert-base-nli-mean-tokens  0.879270  42.581038  418.337442   
1                     all-mpnet-base-v2  0.930934  49.850491  418.360636   

   TOPSIS_Score  Rank  
0      0.990740   1.0  
2      0.872677   2.0  
4      0.604788   3.0  
3      0.199145   4.0  
5      0.144654   5.0  
1      0.059499   6.0  
